In [1]:
pwd

'/Users/aidasaglinskas/Desktop/BC-FacExpr-1.2-fMRI-mainExp/scripts_python'

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image, ImageEnhance,ImageStat
import skvideo
import skvideo.io
import numpy as np
import os
from tqdm import tqdm

In [ ]:
def load_video(fn):
    videodata = skvideo.io.vread(fn)
    return videodata

In [ ]:
def safe_mkdir(dir_name):
    if not os.path.exists(dir_name):
        os.mkdir(dir_name)

In [ ]:
def save_video(videodata,ofn='outvideo.mp4'):
    #skvideo.io.vwrite(ofn, videodata.astype(np.uint8),outputdict={'-codec': 'h264', '-filter:v' : 'fps=29.97'})
    skvideo.io.vwrite(ofn, videodata.astype(np.uint8))

In [ ]:
def pad_frame(img_face,pad_amount=100):
    arr = np.array([np.pad(img_face[:,:,0], (pad_amount, pad_amount), mode='constant'),
    np.pad(img_face[:,:,1], (pad_amount, pad_amount), mode='constant'),
    np.pad(img_face[:,:,2], (pad_amount, pad_amount), mode='constant'),])
    arr = np.swapaxes(arr,0,1)
    arr = np.swapaxes(arr,1,2)
    return arr

In [ ]:
def resize_video(video_data,vid_size_target=(512,512)):
    from PIL import Image, ImageEnhance,ImageStat
    import numpy as np
    
    #vid_size_target = np.array((512,512))
    vid_size_target = np.array(vid_size_target)
    vid_size = np.array((video_data.shape[1],video_data.shape[2]))
    nframes = video_data.shape[0]
    nchannels = video_data.shape[-1]
    video_data_new = np.zeros((nframes,vid_size_target[0],vid_size_target[1],nchannels))
    for f in range(nframes):
        video_data_new[f,:,:,:] = np.array(Image.fromarray(video_data[f,:,:,:]).resize(vid_size_target,Image.ANTIALIAS))

    return video_data_new

In [ ]:
def pad_video(video_data,pad_amount=250,vid_size_target=(512,512)):
    video_data_padded = np.array([pad_frame(video_data[f,:,:,:],pad_amount=pad_amount) for f in range(video_data.shape[0])])
    video_data_padded_resized = resize_video(video_data_padded,vid_size_target=vid_size_target)
    return video_data_padded_resized

In [ ]:
indir = '../stimuli/stimuli_working_dir/stimuli_normed_cb_hue/'
files = [os.path.join(indir,file) for file in os.listdir(indir) if file.endswith('.mp4')]
files.sort()
n = len(files)
print(n)
files[0:5]

In [ ]:
for infn in tqdm(files):
    video_data = load_video(infn)
    for pad in np.arange(0,251,50):
        
        ofdir = f'../stimuli/stimuli_working_dir/stimuli_cropped_normed_crop_{pad}/'
        safe_mkdir(ofdir)
        
        #ofn = infn.replace(indir,ofdir).replace('.mp4',f'_c{pad}.mp4')
        ofn = infn.replace(indir,ofdir)
        video_data_padded = pad_video(video_data,pad_amount=pad,vid_size_target=(512,512))
        save_video(video_data_padded,ofn)